# Bayes ingenuo

Este programa clasifica correos electrónicos como spam o ham utilizando el alrogítmo de bayes ingenuo.

In [1]:
usePackage <- function(p) 
{
  if (!is.element(p, installed.packages()[,1]))
    install.packages(p, repos = "https://cran.itam.mx/")
  suppressPackageStartupMessages(require(p, character.only = TRUE, quietly  = TRUE))
}

In [2]:
usePackage('R.utils')
usePackage('tm')

## Descarga los datos 

In [ ]:
download.mails <- function(url, dir_name, file_name){

  if (!file.exists(dir_name)) {
    dir.create(dir_name)  
  }
  
  download.file(url, destfile=file.path(dir_name, paste0(file_name,".tar.bz2")) )
  bunzip2(file.path(dir_name, paste0(file_name,".tar.bz2")))
  
  untar(file.path(dir_name, paste0(file_name,".tar")), exdir = dir_name)
  
  if (file.exists(file.path(dir_name, paste0(file_name,".tar")))) {
    file.remove(file.path(dir_name, paste0(file_name,".tar")))
  }
  
}

In [ ]:
dir_name <- "data"
file_name <- "easy_ham_2"
url <- "http://spamassassin.apache.org/old/publiccorpus/20030228_easy_ham_2.tar.bz2"

download.mails(url, dir_name, file_name)

In [ ]:
url <- "http://spamassassin.apache.org/old/publiccorpus/20030228_hard_ham.tar.bz2"
file_name <- "hard_ham"

download.mails(url, dir_name, file_name)

In [ ]:
url <- "http://spamassassin.apache.org/old/publiccorpus/20030228_spam_2.tar.bz2"
file_name <- "spam_2"

download.mails(url, dir_name, file_name)

## Preprocesamiento de los correos electrónicos

In [3]:
# Hacemos una función que leea el mensaje del archivo que se le pase como parámetro
# asumimos que el archivo contiene un correo

lee_mensaje <- function(correo) {
  fd <- file(correo, open = "rt")
  lineas <- readLines(fd, warn=FALSE)
  close(fd)
  mensaje <- lineas[seq(which(lineas == "")[1] + 1, length(lineas), 1)]
  return (paste(mensaje, collapse = "\n"))
}

In [4]:
# Creamos variables con los directorios donde se encuentran los datos
dir_name <- "data"
trayectoria_spam     <- file.path(dir_name, "spam_2")
trayectoria_easyham  <- file.path(dir_name, "easy_ham_2")
trayectoria_hardham  <- file.path(dir_name, "hard_ham")

### Spam

In [5]:
# Leemos el directorio donde se encuentran los correos clasificados como spam
archivos_correos_spam <- dir(trayectoria_spam)

# quitamos el guión llamado cmds
archivos_correos_spam <- archivos_correos_spam[which(archivos_correos_spam!="cmds")] #[1:250]

#archivos_correos_spam <- archivos_correos_spam[sample(1:length(archivos_correos_spam))]
#archivos_correos_spam_training <- archivos_correos_spam[1:1000]
#archivos_correos_spam_testing <- archivos_correos_spam[1001:length(archivos_correos_spam)]
archivos_correos_spam <- archivos_correos_spam[sample(1:250)]
archivos_correos_spam_training <- archivos_correos_spam[1:200]
archivos_correos_spam_testing <- archivos_correos_spam[201:250]

todo_spam <- sapply(archivos_correos_spam_training,
                   function(p) lee_mensaje(file.path(trayectoria_spam, p)))
                    
todo_spam <- enc2utf8(todo_spam)

### Easy ham

In [6]:
# Leemos el directorio donde se encuentran los correos clasificados como ham fácilmente identificables
archivos_correos_easy_ham <- dir(trayectoria_easyham)

# quitamos el guión llamado cmds
archivos_correos_easy_ham <- archivos_correos_easy_ham[which(archivos_correos_easy_ham!="cmds")] #[1:250]

#archivos_correos_easy_ham <- archivos_correos_easy_ham[sample(1:length(archivos_correos_easy_ham))]
#archivos_correos_easy_ham_training <- archivos_correos_easy_ham[1:1000]
#archivos_correos_easy_ham_testing <- archivos_correos_easy_ham[1001:length(archivos_correos_easy_ham)]
archivos_correos_easy_ham <- archivos_correos_easy_ham[sample(1:250)]
archivos_correos_easy_ham_training <- archivos_correos_easy_ham[1:200]
archivos_correos_easy_ham_testing <- archivos_correos_easy_ham[201:250]

todo_easy_ham <- sapply(archivos_correos_easy_ham_training,
                    function(p) lee_mensaje(file.path(trayectoria_easyham, p)))

todo_easy_ham <- enc2utf8(todo_easy_ham)

## Preparación de corpus y bolsa de palabras

In [7]:
obtiene_TermDocumentMatrix <- function (vector_correos) {
  control <- list(stopwords = TRUE,
                removePunctuation = TRUE,
                removeNumbers = TRUE,
                minDocFreq = 2)
  corpus <- Corpus(VectorSource(vector_correos))
  return(TermDocumentMatrix(corpus, control))
}

### Spam

In [8]:
spam_TDM <- obtiene_TermDocumentMatrix(todo_spam)

# Crea un data frame que provee el conjunto de caracteristicas de los datos de entrenamiento SPAM
matriz_spam <- as.matrix(spam_TDM)

conteos_spam <- rowSums(matriz_spam)
df_spam <- data.frame(cbind(names(conteos_spam),
                            as.numeric(conteos_spam)),
                      stringsAsFactors = FALSE)
names(df_spam) <- c("terminos", "frecuencia")
df_spam$frecuencia <- as.numeric(df_spam$frecuencia)
ocurrencias_spam <- sapply(1:nrow(matriz_spam),
                          function(i) # Obtiene la proporcion de documentos que contiene cada palabra
                          {
                            length(which(matriz_spam[i, ] > 0)) / ncol(matriz_spam)
                          })
densidad_spam <- df_spam$frecuencia/sum(df_spam$frecuencia,na.rm = TRUE)

df_spam <- transform(df_spam,
                     densidad = densidad_spam,
                     ocurrencias = ocurrencias_spam)

In [9]:
head(df_spam)

,terminos,frecuencia,densidad,ocurrencias
,<chr>,<dbl>,<dbl>,<dbl>
1,able,37,5.498833e-04,0.110
2,address,193,2.868310e-03,0.295
3,also,102,1.515895e-03,0.160
4,always,34,5.052982e-04,0.080
5,audio,2,2.972342e-05,0.010
6,awaiting,2,2.972342e-05,0.010


### Easy ham

In [10]:
easy_ham_TDM <- obtiene_TermDocumentMatrix(todo_easy_ham)

# Crea un data frame que provee el conjunto de caracteristicas de los datos de entrenamiento easy ham
matriz_easy_ham <- as.matrix(easy_ham_TDM)

conteos_easy_ham <- rowSums(matriz_easy_ham)
df_easy_ham <- data.frame(cbind(names(conteos_easy_ham),
                            as.numeric(conteos_easy_ham)),
                      stringsAsFactors = FALSE)
names(df_easy_ham) <- c("terminos", "frecuencia")
df_easy_ham$frecuencia <- as.numeric(df_easy_ham$frecuencia)
ocurrencias_easy_ham <- sapply(1:nrow(matriz_easy_ham),
                           function(i) # Obtiene la proporcion de documentos que contiene cada palabra
                           {
                             length(which(matriz_easy_ham[i, ] > 0)) / ncol(matriz_easy_ham)
                           })
densidad_easy_ham <- df_easy_ham$frecuencia/sum(df_easy_ham$frecuencia,na.rm = TRUE)

df_easy_ham <- transform(df_easy_ham,
                     densidad = densidad_easy_ham,
                     ocurrencias = ocurrencias_easy_ham)

## Cálculo de probabilidad a posteriori

In [11]:
a_posteriori <- function(trayectoria, df_entrenamiento, a_priori = 0.5, c = 1e-6)
{
  mensaje <- lee_mensaje(trayectoria)
  mensaje <- enc2utf8(mensaje)
  mensaje_TDM <- obtiene_TermDocumentMatrix(mensaje)
  conteos_mensaje <- rowSums(as.matrix(mensaje_TDM))

  # Encuentra palabras en data frame de entrenamiento
  mensaje_palabras_comunes <- intersect(names(conteos_mensaje), df_entrenamiento$terminos)
  
  # Ahora sólo aplicamos la clasificación Bayes ingenuo
  if(length(mensaje_palabras_comunes) < 1)
  {
    #return(a_priori * c ^ (length(conteos_mensaje)))
    return(log(a_priori) + (length(conteos_mensaje)) *log(c))
  }
  else
  {
    probabilidades_palabras_comunes <- df_entrenamiento$densidad[match(mensaje_palabras_comunes, df_entrenamiento$terminos)]
    #return(a_priori * prod(probabilidades_palabras_comunes) * c ^ (length(conteos_mensaje) - length(mensaje_palabras_comunes)))
    return(log(a_priori) + sum(log(probabilidades_palabras_comunes)) + log(c) * (length(conteos_mensaje) - length(mensaje_palabras_comunes)))
  }
}

## Clasificación

In [12]:
clasifica_spam <- function(trayectoria, archivos) {

  hard_ham_spam_prueba <- sapply(archivos,
                             function(p) a_posteriori(file.path(trayectoria, p), df_entrenamiento = df_spam))
  hard_ham_ham_prueba <- sapply(archivos,
                            function(p) a_posteriori(file.path(trayectoria, p), df_entrenamiento = df_easy_ham))
  
  return (ifelse(hard_ham_spam_prueba > hard_ham_ham_prueba,
                        TRUE,
                        FALSE))
}

### Hard ham

In [13]:
# Leemos el directorio donde se encuentran los correos clasificados como ham dificlmente identificables
archivos_correos_hard_ham <- dir(trayectoria_hardham)

# quitamos el guión llamado cmds
archivos_correos_hard_ham <- archivos_correos_hard_ham[which(archivos_correos_hard_ham!="cmds")]
archivos_correos_hard_ham<-archivos_correos_hard_ham[sample(50)]

In [14]:
hard_ham_res <- clasifica_spam(trayectoria_hardham, archivos_correos_hard_ham)
easy_ham_res <- clasifica_spam(trayectoria_easyham, archivos_correos_easy_ham_testing)
spam_res     <- clasifica_spam(trayectoria_spam,    archivos_correos_spam_testing)

## Resultados

In [15]:
summary(easy_ham_res)
summary(spam_res)
summary(hard_ham_res)

   Mode   FALSE 
logical      50 

   Mode    TRUE 
logical      50 

   Mode   FALSE    TRUE 
logical       1      49 

# Tarea


## Spam

In [20]:
archivos_correos_spam <- dir(trayectoria_spam) #vector con todos los archivos que estan en ese subdirectorio

# quitamos el guión llamado cmds
archivos_correos_spam <- archivos_correos_spam[which(archivos_correos_spam!="cmds")] #[1:250]

archivos_correos_spam <- archivos_correos_spam[sample(1:length(archivos_correos_spam))]
archivos_correos_spam_training <- archivos_correos_spam[1:200]
archivos_correos_spam_testing <- archivos_correos_spam[201:250]

todo_spam <- sapply(archivos_correos_spam_training,
                    function(p) lee_mensaje(file.path(trayectoria_spam, p))) #hacer una funcion vectorizada y usar una función intermediaria 

todo_spam <- enc2utf8(todo_spam)


In [21]:
spam_TDM <- obtiene_TermDocumentMatrix(todo_spam)

# Crea un data frame que provee el conjunto de caracteristicas de los datos de entrenamiento SPAM
matriz_spam <- as.matrix(spam_TDM) #convertirla en matriz de R

conteos_spam <- rowSums(matriz_spam)
df_spam <- data.frame(cbind(names(conteos_spam),
                            as.numeric(conteos_spam)),
                      stringsAsFactors = FALSE)
names(df_spam) <- c("terminos", "frecuencia")
df_spam$frecuencia <- as.numeric(df_spam$frecuencia)
ocurrencias_spam <- sapply(1:nrow(matriz_spam),
                           function(i) # Obtiene la proporcion de documentos que contiene cada palabra
                           {
                             length(which(matriz_spam[i, ] > 0)) / ncol(matriz_spam)
                           })
densidad_spam <- df_spam$frecuencia/sum(df_spam$frecuencia,na.rm = TRUE) #na.rm quita na

df_spam <- transform(df_spam,
                     densidad = densidad_spam,
                     ocurrencias = ocurrencias_spam)



## Easy Ham

In [47]:
archivos_correos_easy_ham <- dir(trayectoria_easyham)

# quitamos el guión llamado cmds
archivos_correos_easy_ham <- archivos_correos_easy_ham[which(archivos_correos_easy_ham!="cmds")] #[1:250]

archivos_correos_easy_ham <- archivos_correos_easy_ham[sample(1:length(archivos_correos_easy_ham))]
archivos_correos_easy_ham_training <- archivos_correos_easy_ham[1:200]
archivos_correos_easy_ham_testing <- archivos_correos_easy_ham[201:250]

todo_easy_ham <- sapply(archivos_correos_easy_ham_training,
                        function(p) lee_mensaje(file.path(trayectoria_easyham, p)))

todo_easy_ham <- enc2utf8(todo_easy_ham)

In [48]:
easy_ham_TDM <- obtiene_TermDocumentMatrix(todo_easy_ham)

# Crea un data frame que provee el conjunto de caracteristicas de los datos de entrenamiento easy ham

matriz_easy_ham <- as.matrix(easy_ham_TDM)

conteos_easy_ham <- rowSums(matriz_easy_ham)
df_easy_ham <- data.frame(cbind(names(conteos_easy_ham),
                                as.numeric(conteos_easy_ham)),
                          stringsAsFactors = FALSE)
names(df_easy_ham) <- c("terminos", "frecuencia")
df_easy_ham$frecuencia <- as.numeric(df_easy_ham$frecuencia)
ocurrencias_easy_ham <- sapply(1:nrow(matriz_easy_ham),
                               function(i) # Obtiene la proporcion de documentos que contiene cada palabra
                               {
                                 length(which(matriz_easy_ham[i, ] > 0)) / ncol(matriz_easy_ham)
                               })
densidad_easy_ham <- df_easy_ham$frecuencia/sum(df_easy_ham$frecuencia,na.rm = TRUE)

df_easy_ham <- transform(df_easy_ham,
                         densidad = densidad_easy_ham,
                         ocurrencias = ocurrencias_easy_ham)


## Hard Ham

In [49]:


archivos_correos_hard_ham <- dir(trayectoria_hardham)
archivos_correos_hard_ham <- archivos_correos_hard_ham[which(archivos_correos_hard_ham!="cmds")] #[1:250]

archivos_correos_hard_ham <- archivos_correos_hard_ham[sample(1:length(archivos_correos_hard_ham))]
archivos_correos_hard_ham_training <- archivos_correos_hard_ham[1:200]
archivos_correos_hard_ham_testing <- archivos_correos_hard_ham[201:length(archivos_correos_hard_ham)]

trayectoria_hardham  <- file.path(dir_name, "hard_ham")

todo_hard_ham <- sapply(archivos_correos_hard_ham_training,
                        function(p) lee_mensaje(file.path(trayectoria_hardham, p)))


todo_hard_ham <- enc2utf8(todo_hard_ham)





In [50]:
hard_ham_TDM <- obtiene_TermDocumentMatrix(todo_hard_ham)

matriz_hard_ham <- as.matrix(hard_ham_TDM)

conteos_hard_ham <- rowSums(matriz_hard_ham)
df_hard_ham <- data.frame(cbind(names(conteos_hard_ham),
                                as.numeric(conteos_hard_ham)),
                          stringsAsFactors = FALSE)
names(df_hard_ham) <- c("terminos", "frecuencia")
df_hard_ham$frecuencia <- as.numeric(df_hard_ham$frecuencia)
ocurrencias_hard_ham <- sapply(1:nrow(matriz_hard_ham),
                               function(i)
                               {
                                 length(which(matriz_hard_ham[i, ] > 0)) / ncol(matriz_hard_ham)
                               })
densidad_hard_ham <- df_hard_ham$frecuencia/sum(df_hard_ham$frecuencia,na.rm = TRUE)

df_hard_ham <- transform(df_hard_ham,
                         densidad = densidad_hard_ham,
                         ocurrencias = ocurrencias_hard_ham)


## Training

In [51]:
hard_ham_res2 <- clasifica_spam(trayectoria_hardham, archivos_correos_hard_ham_testing)
easy_ham_res2 <- clasifica_spam(trayectoria_easyham, archivos_correos_easy_ham)
spam_res2     <- clasifica_spam(trayectoria_spam,    archivos_correos_spam_testing)

In [52]:
summary(hard_ham_res2)
summary(easy_ham_res2)
summary(spam_res2)

   Mode   FALSE    TRUE 
logical      21      29 

   Mode   FALSE    TRUE 
logical    1385      15 

   Mode   FALSE    TRUE 
logical       5      45 

## Argumento Máximo

In [53]:
install.packages('ramify', dependencies = TRUE, repos='http://cran.rstudio.com/')


The downloaded binary packages are in
	/var/folders/9j/5g2kcd8x5y1gth6pxchhjdp80000gn/T//RtmpYs7l6l/downloaded_packages


In [54]:
library(ramify)

In [55]:
clasifica_spam_max <- function (trayectoria, archivos) {
    spam_prueba <- sapply(archivos,
                        function (p) a_posteriori(file.path(trayectoria, p), df_entrenamiento = df_spam))
    ham_prueba <- sapply(archivos,
                        function(p)  a_posteriori(file.path(trayectoria, p), df_entrenamiento = df_easy_ham))
    hard_ham_prueba <- sapply(archivos,
                        function (p) a_posteriori(file.path(trayectoria, p), df_entrenamiento = df_hard_ham))
    x=matrix(c(ham_prueba, hard_ham_prueba, spam_prueba), nrow=length(ham_prueba))
    return (as.factor(argmax(x)))
}

In [56]:

easy_ham_max_res2 <- clasifica_spam_max(trayectoria_easyham, archivos_correos_easy_ham_testing)
hard_ham_max_res2 <- clasifica_spam_max(trayectoria_hardham, archivos_correos_hard_ham_testing)
spam_res_max2    <- clasifica_spam_max(trayectoria_spam,    archivos_correos_spam_testing)

In [57]:
summary(easy_ham_max_res2)
summary(hard_ham_max_res2)
summary(spam_res_max2)

1  2 
47  3

1  2  3 
 9 38  3

1  2  3 
 5  2 43

El objetivo de este estudio es ver si realmente hay una diferencia entre entrenar el modelo con los correos de esay_ham o con el hard_ham y easy_harm. Para poder tener la misma significancia solo vamos a entrenar los modelos con 200 correos y hacer la prueba con 50 correos.

Los resultados obtenidos en este estudio, a primer instante, da la impresión de ser muy diferentes. Cuando uno se fija en el primer experimento (aquel en el que se toma easy_ham), tenemos que logra clasificar los correos de forma muy eficiente teniendo un solo error en el spam. Por el otro lado, cuando usamos el entrenamiento en el hard_harm  y easy_harm obtenemos a primera impresión resultados malos, pero una vez que aplicamos la funcion argmax, nos damos cuenta que realmente no es nada malo. Vemos que a primera instancia que el mayor error se genera en los correos hard y cuando aplicamos la funcion argmax, nos damos cuenta que la mayoría de los errores se van a easy_ham que en realidad no es incorrecto. Si solo tuvieramos dos clases: ham o spam, entonces los resultados serían muy parecidos.

A este estudio le faltaría agragar más correos hard_ham para así poder aumentar nuestro modelo de aprendizaje, agregar más palabras, y poder hacer el testing con más datos para poder obtener resultados más significativos y poder determinar puntualmente si realmente existe una gran diferencia con entrenar los modelos de forma diferente.